In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install imgaug opencv-python-headless numpy


In [ ]:
import os
import cv2
import numpy as np
from imgaug import augmenters as iaa

# Veri artırma sekansını tanımlayalım
augmentation_sequence = iaa.Sequential([
    iaa.Affine(rotate=(-25, 25)),  # Rotasyon
    iaa.Multiply((0.8, 1.2)),  # Parlaklık değişimi
    iaa.AddToHueAndSaturation((-20, 20)),  # Renk doygunluğu ve tonu değişimi
    iaa.Fliplr(0.5),  # Yatay çevirme
    iaa.CropAndPad(percent=(-0.2, 0.2)),  # Kırpma ve pedleme
    iaa.GaussianBlur(sigma=(0, 3.0)),  # Gaussian bulanıklık
    iaa.MultiplyElementwise((0.8, 1.2)),  # Renk jitter
    iaa.MixUpAlpha((0.2, 0.8), iaa.BlendAlphaElementwise(iaa.Multiply((0.8, 1.2))))  # MixUp
])

def mosaic(images):

    h, w = images[0].shape[:2]
    mosaic_image = np.zeros((2*h, 2*w, 3), dtype=images[0].dtype)
    mosaic_image[0:h, 0:w] = images[0]
    mosaic_image[0:h, w:2*w] = images[1]
    mosaic_image[h:2*h, 0:w] = images[2]
    mosaic_image[h:2*h, w:2*w] = images[3]
    return mosaic_image

def augment_images(input_dir, output_dir, augmentations_per_image=5):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = []
    for filename in os.listdir(input_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)
            images.append(image)

    for idx, image in enumerate(images):
        for i in range(augmentations_per_image):
            augmented_image = augmentation_sequence(image=image)

            # Mosaic işlemi için rastgele 3 diğer görüntü seç
            if len(images) >= 4:
                selected_images = [image]
                while len(selected_images) < 4:
                    rand_img = images[np.random.randint(len(images))]
                    if rand_img is not image:
                        selected_images.append(rand_img)
                mosaic_image = mosaic(selected_images)
                augmented_image = augmentation_sequence(image=mosaic_image)

            output_filename = f"image_{idx}_{i}.png"
            output_path = os.path.join(output_dir, output_filename)
            cv2.imwrite(output_path, augmented_image)

# Girdi ve çıktı klasörlerini tanımlayalım
input_directory = '/content/drive/MyDrive/FracatlasORJ2/train/Fractured'
output_directory = '/content/drive/MyDrive/AUGMNT'

# Veri artırma işlemini gerçekleştirelim
augment_images(input_directory, output_directory)
